# Train Music Transformer
Since Fri. Feb. 25th, 2022

Set up training in colab


## Setup


### Ipython



In [1]:
%load_ext autoreload
%autoreload 2



### Colab



In [2]:
import os
import sys

import torch


if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    ! pip3 install sty icecream music21 transformers datasets

    path = os.path.join('drive', 'My Drive', 'Research', 'Music with NLP', 'Symbolic-Music-Generation')
    sys.path.append(path)
    ! ls "{path}"


    import time, os
    os.environ['TZ'] = 'US/Eastern'
    time.tzset()

    if torch.cuda.is_available():
        %env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

if torch.cuda.is_available():
    ! nvidia-smi


from icecream import ic

from musicnlp.util import *

ic(PATH_BASE, DIR_PROJ, PKG_NM)



Mounted at /content/drive
     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 311 kB 52.0 MB/s 
     |████████████████████████████████| 212 kB 68.7 MB/s 
     |████████████████████████████████| 1.1 MB 45.8 MB/s 
     |████████████████████████████████| 77 kB 7.3 MB/s 
     |████████████████████████████████| 136 kB 70.8 MB/s 
     |████████████████████████████████| 596 kB 51.9 MB/s 
     |████████████████████████████████| 6.6 MB 53.6 MB/s 
     |████████████████████████████████| 895 kB 47.2 MB/s 
     |████████████████████████████████| 271 kB 69.5 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 73.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
models	musicnlp  notebook
env: PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128
Fri Apr 15 18:44:28 2022       
+------------------------

ic| PATH_BASE: 'drive/My Drive/Research/Music with NLP'
    DIR_PROJ: 'Symbolic-Music-Generation'
    PKG_NM: 'musicnlp'


('drive/My Drive/Research/Music with NLP',
 'Symbolic-Music-Generation',
 'musicnlp')

### code

In [3]:
from musicnlp.util import *
from musicnlp.models import train


seed = config('random-seed')



## Prep for training


In [4]:
dnm_909 = 'musicnlp music extraction, dnm=POP909, n=909, meta={mode=melody, prec=5, th=1}, 2022-04-10_12-51-01'
dnm_lmd = 'musicnlp music extraction, dnm=LMD-cleaned-subset, ' \
          'n=10269, meta={mode=melody, prec=5, th=1}, 2022-04-10_19-49-52'
dnms = [dnm_909, dnm_lmd]

md_nm = 'reformer'
# md_sz = 'tiny'
md_sz = 'base'


if md_nm != 'reformer':
    import transformers
    transformers.set_seed(seed)


if md_sz == 'debug':
    n = 8
    train_args = dict(
        per_device_train_batch_size=4,
        # save_strategy='no',
        save_strategy='epoch',
        num_train_epochs=64,
    )
    my_train_args = dict(
        save_epochs=16
    )
elif md_sz == 'tiny':
    n = 64
    train_args = dict(
        per_device_train_batch_size=32,
        save_strategy='epoch',
        num_train_epochs=64,
    )
    my_train_args = dict(
        save_epochs=16
)
else:
    n = None
    train_args = dict(
        num_train_epochs=32,
        per_device_train_batch_size=32,
    )
    my_train_args = dict(
        fp16=torch.cuda.is_available(),
        logging_strategy='epoch',
        save_epochs=2,
        disable_tqdm=False
    )

mdl, tokenizer, trainer = train.get_all_setup(
    model_name=md_nm, model_size=md_sz, dataset_names=dnms, n_sample=n, dataset_seed=seed,
    train_args=train_args, my_train_args=my_train_args
)



Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

## Train


Check log and tensorboard files written



In [ ]:
# trainer.train()
checkpoint_path = os.path.join(PATH_BASE, DIR_PROJ, DIR_MDL, 'reformer', '2022-04-15_13-42-56', 'checkpoint-2058')
trainer.train(checkpoint_path)

trainer.save_model(os.path.join(trainer.args.output_dir, 'trained'))


2022-04-15 18:46:11|[ReformerModelWithLMHead Training]::on_train_begin::train.py:180, INFO - Training started with model {model name: ReformerModelWithLMHead, max length: 2048, axial_pos_shape: (32, 64), n_layer: 12, hidden_size: 768, ff_size: 3072, attention_shape: 12x64, parameter_count: 42.1M}, {
    "hash_seed": null,
    "vocab_size": 1437,
    "attention_head_size": 64,
    "hidden_size": 768,
    "num_attention_heads": 12,
    "num_hashes": 2,
    "num_hidden_layers": 6,
    "num_buckets": null,
    "lsh_attn_chunk_length": 64,
    "local_attn_chunk_length": 64,
    "lsh_num_chunks_after": 0,
    "lsh_num_chunks_before": 1,
    "local_num_chunks_after": 0,
    "local_num_chunks_before": 1,
    "hidden_act": "relu",
    "feed_forward_size": 3072,
    "hidden_dropout_prob": 0.05,
    "lsh_attention_probs_dropout_prob": 0.0,
    "local_attention_probs_dropout_prob": 0.05,
    "max_position_embeddings": 2048,
    "initializer_range": 0.02,
    "layer_norm_eps": 1e-12,
    "axial_pos

config.num_buckets is not set. Setting config.num_buckets to 64...
config.num_buckets is not set. Setting config.num_buckets to 64...
config.num_buckets is not set. Setting config.num_buckets to 64...


2022-04-15 19:43:14|[ReformerModelWithLMHead Training]::_log::train.py:249, INFO - {step:  2401/10976, epoch:  7.000/32, train/learning_rate: 2.873e-04, train/loss: 1.1973, train/ntp_acc: 39.46, train/ikr: 74.59}
2022-04-15 19:44:12|[ReformerModelWithLMHead Training]::_log::train.py:249, INFO - {step:  2401/10976, epoch:  7/32, eval/loss: 1.347, eval/ntp_acc: 38.67, eval/ikr: 65.98}
2022-04-15 20:42:19|[ReformerModelWithLMHead Training]::_log::train.py:249, INFO - {step:  2744/10976, epoch:  8.000/32, train/learning_rate: 2.799e-04, train/loss: 1.361, train/ntp_acc:  34.6, train/ikr: 70.44}
2022-04-15 20:43:22|[ReformerModelWithLMHead Training]::_log::train.py:249, INFO - {step:  2744/10976, epoch:  8/32, eval/loss: 1.2886, eval/ntp_acc: 39.77, eval/ikr: 65.24}
2022-04-15 21:43:15|[ReformerModelWithLMHead Training]::_log::train.py:249, INFO - {step:  3087/10976, epoch:  9.000/32, train/learning_rate: 2.710e-04, train/loss: 0.8757, train/ntp_acc: 48.94, train/ikr: 64.99}
2022-04-15 21:4

In [ ]:
ic(trainer.args.output_dir)
ic(os.listdir(trainer.args.output_dir))

